## 배치 트랜스포머 - 배치 예측

라이브러리 준비

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import awswrangler as wr

In [4]:
import os
import sagemaker
import boto3
from dotenv import load_dotenv
load_dotenv()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/dante/Library/Application Support/sagemaker/config.yaml


True

SageMaker 세션 및 역할 설정

In [5]:
boto3_session = boto3.Session(profile_name='awstutor')
sagemaker_session = sagemaker.Session(boto_session=boto3_session)
role = os.environ.get('SAGEMAKER_EXECUTION_ROLE_ARN')

S3 경로 설정

In [6]:
bucket_name = 'dante-sagemaker'
project_name = 'mushroom-classification'

In [7]:
input_path = f's3://{bucket_name}/{project_name}/input'
output_path = f's3://{bucket_name}/{project_name}/output'
model_path = f's3://{bucket_name}/{project_name}/model'
asset_path = f's3://{bucket_name}/{project_name}/asset'
test_path = f'{input_path}/test/test.csv'

In [8]:
import io
import pickle
from sagemaker.amazon.common import write_numpy_to_dense_tensor

# RecordIO 형식으로 데이터를 변환하고 S3에 저장하는 헬퍼 함수
def convert_to_recordio_protobuf(df, label_column, s3_path):
    # 레이블과 특성 분리
    labels = df[label_column].values
    features = df.drop(columns=[label_column]).values
    
    # RecordIO-Protobuf 형식으로 변환
    buf = io.BytesIO()
    write_numpy_to_dense_tensor(buf, features, labels)
    buf.seek(0)
    
    # S3에 업로드
    wr.s3.upload(local_file=buf, path=s3_path, boto3_session=boto3_session)
    print(f"데이터가 {s3_path}에 RecordIO-Protobuf 형식으로 저장되었습니다.")

테스트 데이터셋을 입력 데이터로 재구성

In [9]:
# S3에서 테스트 데이터를 읽어옵니다
test_df = wr.s3.read_csv(test_path, header=None, boto3_session=boto3_session)

# S3에서 특성 목록을 읽어옵니다
features = wr.s3.read_csv(os.path.join(asset_path, 'features.csv'), header=None, boto3_session=boto3_session).values.flatten()

# 테스트 데이터프레임에 열 이름을 지정합니다
test_df.columns = ['poisonous'] + list(features)

In [10]:
# 인코더 파일 경로 설정
feature_encoders_filepath = 'assets/feature_encoders.pkl'
label_encoder_filepath = 'assets/label_encoder.pkl'

# S3에서 인코더 파일 다운로드
wr.s3.download(os.path.join(asset_path, 'label_encoder.pkl'), label_encoder_filepath, boto3_session=boto3_session)
wr.s3.download(os.path.join(asset_path, 'feature_encoders.pkl'), feature_encoders_filepath, boto3_session=boto3_session)

# 인코더 파일 로드
label_encoder = pickle.load(open(label_encoder_filepath, 'rb'))
feature_encoders = pickle.load(open(feature_encoders_filepath, 'rb'))

In [11]:
# test_df에 포함되지 않은 레이블도 처리하고 업데이트된 인코더를 S3에 업로드하도록 수정
def safe_transform_and_update(encoder, data, feature_name):
    unique_values = data.unique()
    new_classes = set(unique_values) - set(encoder.classes_)
    
    if new_classes:
        encoder.classes_ = np.concatenate([encoder.classes_, list(new_classes)])
        feature_encoders[feature_name] = encoder  # 인코더 업데이트
    
    return encoder.transform(data)

In [12]:
# 테스트 데이터 인코딩
test_df[features] = test_df[features].apply(lambda x: safe_transform_and_update(feature_encoders[x.name], x, x.name))
test_df['poisonous'] = label_encoder.transform(test_df['poisonous'])
test_df.head()

,poisonous,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0,5,3,3,1,5,1,0,0,5,1,1,2,2,7,6,0,2,1,4,3,4,0
1,0,3,3,4,0,5,1,1,1,10,0,1,2,0,7,4,0,2,1,0,7,4,2
2,1,2,2,4,1,6,1,0,1,5,0,3,2,2,7,7,0,2,1,4,2,3,5
3,1,2,2,4,0,2,1,0,1,0,1,0,2,1,7,6,0,2,1,0,7,4,0
4,1,2,2,2,0,8,1,0,1,0,1,0,1,1,6,7,0,2,1,0,7,4,4


In [13]:
# 테스트 데이터를 RecordIO-Protobuf 형식으로 변환하여 S3에 저장
test_new_path = f"{input_path}/test/test.recordio"
convert_to_recordio_protobuf(test_df, 'poisonous', test_new_path)

데이터가 s3://dante-sagemaker/mushroom-classification/input/test/test.recordio에 RecordIO-Protobuf 형식으로 저장되었습니다.


In [14]:
# 업데이트된 인코더를 S3에 업로드
with open(feature_encoders_filepath, 'wb') as f:
    pickle.dump(feature_encoders, f)

wr.s3.upload(feature_encoders_filepath, os.path.join(asset_path, 'feature_encoders.pkl'), boto3_session=boto3_session)

배치 트랜스포머 준비

In [15]:
# 모델 아티팩트에서 estimator 가져오기
training_job_name = 'mushroom-classification-2024-07-30-23-07-32-763'
# 모델 아티팩트 경로
model_artifact = sagemaker_session.sagemaker_client.describe_training_job(
    TrainingJobName=training_job_name
)['ModelArtifacts']['S3ModelArtifacts']
model_artifact

's3://dante-sagemaker/mushroom-classification/output/mushroom-classification-2024-07-30-23-07-32-763/output/model.tar.gz'

In [16]:
# XGBoost 모델 생성
from sagemaker.xgboost import XGBoostModel
estimator = XGBoostModel(
    model_data=model_artifact,
    role=role,  # 이전에 정의한 IAM 역할
    sagemaker_session=sagemaker_session,
    framework_version='1.7-1',  # XGBoost 버전
    py_version='py3',
)

In [17]:
# XGBoost 모델을 사용하여 배치 변환을 위한 트랜스포머 객체 생성
xgb_transformer = estimator.transformer(
    instance_count=1,  # 사용할 인스턴스 수
    instance_type='ml.m5.xlarge',  # 사용할 인스턴스 유형
    strategy='MultiRecord',  # 여러 레코드를 한 번에 처리하는 전략
    assemble_with='Line',  # 출력을 줄 단위로 조립
    output_path=output_path,  # 변환 결과를 저장할 S3 경로
)

배치 트랜스포머 실행

In [18]:
xgb_transformer.transform(test_new_path, content_type='application/x-recordio-protobuf')
xgb_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-08-01-11-28-42-189


.............................[2024-08-01:11:33:31:INFO] No GPUs detected (normal if no gpus installed)
[2024-08-01:11:33:31:INFO] No GPUs detected (normal if no gpus installed)
[2024-08-01:11:33:31:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
[2024-08-01:11:33:31:INFO] No GPUs detected (normal if no gpus installed)
[2024-08-01:11:33:31:INFO] No GPUs detected (normal if no gpus installed)
[2024-08-01:11:33:31:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|

예측 평가

In [19]:
# .out 확장자로 끝나는 파일 조회
out_files = wr.s3.list_objects(output_path, suffix='.out', boto3_session=boto3_session)

print("S3에서 발견된 .out 파일들:")
for file in out_files:
    print(file)

S3에서 발견된 .out 파일들:
s3://dante-sagemaker/mushroom-classification/output/test.recordio.out


In [20]:
df = wr.s3.read_csv(out_files[0], header=None, boto3_session=boto3_session)
df.columns = ['prediction']

# 결과 출력
print(df.head())
print(f"총 예측 수: {len(df)}")
print(f"양성 예측 수: {df[df['prediction'] >= 0.5].shape[0]}")
print(f"음성 예측 수: {df[df['prediction'] < 0.5].shape[0]}")


   prediction
0    0.004635
1    0.133454
2    0.975551
3    0.995080
4    0.997364
총 예측 수: 1625
양성 예측 수: 782
음성 예측 수: 843


In [21]:
from sklearn.metrics import classification_report

# 실제 레이블 가져오기
actual_labels = test_df['poisonous']

# 예측 레이블 생성 (0.5를 임계값으로 사용)
predicted_labels = (df['prediction'] >= 0.5).astype(int)

# 분류 보고서 생성
report = classification_report(actual_labels, predicted_labels)

print("분류 보고서:")
print(report)


분류 보고서:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       842
           1       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625

